In [ ]:
#Basic Client
import socket
import threading


def handler(sock):
    
    while True:
        data = sock.recv(1024)
        print('\nreceived: ',data.decode())
        

sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
host = input("Server Address(default=127.0.0.1): ")
if host == '':
    host = '127.0.0.1'
sock.connect((host, 2500))
cThread = threading.Thread(target=handler, args=(sock,))
cThread.daemon = True
cThread.start()

while True:
    msg = input('> ')
    sock.send(msg.encode())


received:  ASD

received:  DDD


In [7]:

#완성된 코드
import socket
import threading
import tkinter as tk

from tkinter import ttk, messagebox

class ChatClient:
    def __init__(self, master):
        self.master = master
        self.master.title("NA_KakaoTalk")
        self.master.geometry("350x500")
        self.master.configure(bg="#b2c7d9")

        self.nickname = None
        self.sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)

        self.create_login_frame()
        self.create_chat_frame()

        self.current_frame = self.login_frame
        self.show_frame(self.login_frame)

    def create_login_frame(self):
        self.login_frame = tk.Frame(self.master, bg="#b2c7d9")
        
        tk.Label(self.login_frame, text="서버 주소:", bg="#b2c7d9").pack(pady=5)
        self.server_entry = tk.Entry(self.login_frame)
        self.server_entry.insert(0, '127.0.0.1')
        self.server_entry.pack(pady=5)

        tk.Label(self.login_frame, text="사용자 이름:", bg="#b2c7d9").pack(pady=5)
        self.username_entry = tk.Entry(self.login_frame)
        self.username_entry.pack(pady=5)

        tk.Label(self.login_frame, text="비밀번호:", bg="#b2c7d9").pack(pady=5)
        self.password_entry = tk.Entry(self.login_frame, show='*')
        self.password_entry.pack(pady=5)

        self.login_button = ttk.Button(self.login_frame, text="로그인", command=self.login)
        self.login_button.pack(pady=10)

        self.register_button = ttk.Button(self.login_frame, text="회원가입", command=self.register)
        self.register_button.pack(pady=5)

    def create_chat_frame(self):
        self.chat_frame = tk.Frame(self.master, bg="#b2c7d9")

        self.chat_canvas = tk.Canvas(self.chat_frame, bg="#b2c7d9", highlightthickness=0)
        self.chat_canvas.pack(expand=True, fill='both', padx=10, pady=10)

        self.scrollbar = ttk.Scrollbar(self.chat_canvas, orient="vertical", command=self.chat_canvas.yview)
        self.scrollbar.pack(side="right", fill="y")

        self.chat_canvas.configure(yscrollcommand=self.scrollbar.set)

        self.message_frame = tk.Frame(self.chat_canvas, bg="#b2c7d9")
        self.chat_canvas.create_window((50, 0), window=self.message_frame, anchor="nw")

        self.input_frame = tk.Frame(self.chat_frame, bg="#b2c7d9")
        self.input_frame.pack(side=tk.BOTTOM, fill=tk.X, padx=10, pady=10)

        self.message_entry = tk.Entry(self.input_frame, bg="white", fg="black")
        self.message_entry.pack(side=tk.LEFT, expand=True, fill=tk.X, padx=(0, 5))
        self.message_entry.bind("<Return>", self.send_message)

        self.send_button = ttk.Button(self.input_frame, text="전송", command=self.send_message)
        self.send_button.pack(side=tk.RIGHT)

        self.message_frame.bind("<Configure>", self.on_frame_configure)

    def show_frame(self, frame):
        if self.current_frame:
            self.current_frame.pack_forget()
        frame.pack(expand=True, fill='both')
        self.current_frame = frame

    def on_frame_configure(self, event):
        self.chat_canvas.configure(scrollregion=self.chat_canvas.bbox("all"))

    def connect_to_server(self):
        host = self.server_entry.get()
        try:
            self.sock.connect((host, 2500))
            return True
        except Exception as e:
            messagebox.showerror("연결 오류", f"서버 연결 실패: {str(e)}")
            return False
    def connect_to_server(self):
        host = self.server_entry.get()
        try:
            if self.sock:
                self.sock.close()
            self.sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
            self.sock.connect((host, 2500))
            return True
        except Exception as e:
            messagebox.showerror("연결 오류", f"서버 연결 실패: {str(e)}")
            return False
        
    def login(self):
        if not self.connect_to_server():
            return

        username = self.username_entry.get()
        password = self.password_entry.get()
        self.sock.send(f"LOGIN:{username}:{password}".encode())
        response = self.sock.recv(1024).decode()
        if response == "LOGIN_SUCCESS":
            self.nickname = username
            self.show_frame(self.chat_frame)
            self.display_message(f"{username}님으로 로그인했습니다.", 'left', 'white')
            receive_thread = threading.Thread(target=self.receive_messages)
            receive_thread.daemon = True
            receive_thread.start()
        else:
            messagebox.showerror("로그인 실패", "아이디 또는 비밀번호가 올바르지 않습니다.")
            self.sock.close()
            self.sock = None

    def register(self):
        username = self.username_entry.get()
        password = self.password_entry.get()
        
        if not username or not password:
            messagebox.showerror("입력 오류", "사용자 이름과 비밀번호를 모두 입력해주세요.")
            return
        
        confirm = messagebox.askyesno("계정 생성 확인", f"'{username}' 계정을 생성하시겠습니까?")
        if not confirm:
            return
        
        if not self.connect_to_server():
            return

        self.sock.send(f"REGISTER:{username}:{password}".encode())
        response = self.sock.recv(1024).decode()
        if response == "REGISTER_SUCCESS":
            messagebox.showinfo("회원가입 성공", "회원가입이 완료되었습니다. 로그인해주세요.")
        else:
            messagebox.showerror("회원가입 실패", "이미 존재하는 사용자명입니다.")
        self.sock.close()
        self.sock = None

    def send_message(self, event=None):
        message = self.message_entry.get()
        if message and self.nickname:
            try:
                self.sock.send(message.encode())
                self.message_entry.delete(0, tk.END)
            except:
                self.display_message("메시지 전송 실패", 'left', 'red')

    def receive_messages(self):
        while True:
            try:
                data = self.sock.recv(1024)
                if data:
                    message = data.decode()
                    if message.startswith(f"{self.nickname}:"):
                        self.display_message(message, 'right', '#fee500')
                    else:
                        self.display_message(message, 'left', 'white')
            except:
                self.display_message("서버와의 연결이 끊겼습니다.", 'left', 'red')
                break

    def display_message(self, message, align, bg_color):
        frame = tk.Frame(self.message_frame, bg="#b2c7d9")
        frame.pack(side=tk.TOP, fill=tk.X, padx=2, pady=2)

        if align == 'left':
            frame.pack_configure(anchor='w')
            msg_label = tk.Label(frame, text=message, bg=bg_color, fg="black", wraplength=200, justify=tk.LEFT, padx=5, pady=5)
            msg_label.pack(side=tk.LEFT)
        else:
            frame.pack_configure(anchor='e')
            msg_label = tk.Label(frame, text=message, bg=bg_color, fg="black", wraplength=200, justify=tk.RIGHT, padx=5, pady=5)
            msg_label.pack(side=tk.RIGHT)

        self.chat_canvas.update_idletasks()
        self.chat_canvas.yview_moveto(1.0)

        
if __name__ == "__main__":
    root = tk.Tk()
    client = ChatClient(root)
    root.mainloop()


Exception in thread Thread-15 (receive_messages):
Traceback (most recent call last):
  File "C:\Users\fun67\AppData\Local\Temp\ipykernel_16900\4157794496.py", line 163, in receive_messages
  File "C:\Users\fun67\AppData\Local\Temp\ipykernel_16900\4157794496.py", line 169, in display_message
  File "c:\Users\fun67\anaconda3\Lib\tkinter\__init__.py", line 3224, in __init__
    Widget.__init__(self, master, 'frame', cnf, {}, extra)
  File "c:\Users\fun67\anaconda3\Lib\tkinter\__init__.py", line 2659, in __init__
    self.tk.call(
RuntimeError: main thread is not in main loop

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "c:\Users\fun67\anaconda3\Lib\threading.py", line 1073, in _bootstrap_inner
    self.run()
  File "c:\Users\fun67\anaconda3\Lib\threading.py", line 1010, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\fun67\AppData\Local\Temp\ipykernel_16900\4157794496.py", line 165, in receive_message

In [ ]:

#시간기능 추가가
import socket
import threading
import tkinter as tk
import datetime

from tkinter import ttk, messagebox

class ChatClient:
    def __init__(self, master):
        self.master = master
        self.master.title("NA_KakaoTalk")
        self.master.geometry("350x500")
        self.master.configure(bg="#b2c7d9")

        self.nickname = None
        self.sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)

        self.create_login_frame()
        self.create_chat_frame()

        self.current_frame = self.login_frame
        self.show_frame(self.login_frame)

    def create_login_frame(self):
        self.login_frame = tk.Frame(self.master, bg="#b2c7d9")
        
        tk.Label(self.login_frame, text="서버 주소:", bg="#b2c7d9").pack(pady=5)
        self.server_entry = tk.Entry(self.login_frame)
        self.server_entry.insert(0, '127.0.0.1')
        self.server_entry.pack(pady=5)

        tk.Label(self.login_frame, text="사용자 이름:", bg="#b2c7d9").pack(pady=5)
        self.username_entry = tk.Entry(self.login_frame)
        self.username_entry.pack(pady=5)

        tk.Label(self.login_frame, text="비밀번호:", bg="#b2c7d9").pack(pady=5)
        self.password_entry = tk.Entry(self.login_frame, show='*')
        self.password_entry.pack(pady=5)

        self.login_button = ttk.Button(self.login_frame, text="로그인", command=self.login)
        self.login_button.pack(pady=10)

        self.register_button = ttk.Button(self.login_frame, text="회원가입", command=self.register)
        self.register_button.pack(pady=5)

    def create_chat_frame(self):
        self.chat_frame = tk.Frame(self.master, bg="#b2c7d9")

        self.chat_canvas = tk.Canvas(self.chat_frame, bg="#b2c7d9", highlightthickness=0)
        self.chat_canvas.pack(expand=True, fill='both', padx=10, pady=10)

        self.scrollbar = ttk.Scrollbar(self.chat_canvas, orient="vertical", command=self.chat_canvas.yview)
        self.scrollbar.pack(side="right", fill="y")

        self.chat_canvas.configure(yscrollcommand=self.scrollbar.set)

        self.message_frame = tk.Frame(self.chat_canvas, bg="#b2c7d9")
        self.chat_canvas.create_window((50, 0), window=self.message_frame, anchor="nw")

        self.input_frame = tk.Frame(self.chat_frame, bg="#b2c7d9")
        self.input_frame.pack(side=tk.BOTTOM, fill=tk.X, padx=10, pady=10)

        self.message_entry = tk.Entry(self.input_frame, bg="white", fg="black")
        self.message_entry.pack(side=tk.LEFT, expand=True, fill=tk.X, padx=(0, 5))
        self.message_entry.bind("<Return>", self.send_message)

        self.send_button = ttk.Button(self.input_frame, text="전송", command=self.send_message)
        self.send_button.pack(side=tk.RIGHT)

        self.message_frame.bind("<Configure>", self.on_frame_configure)

    def show_frame(self, frame):
        if self.current_frame:
            self.current_frame.pack_forget()
        frame.pack(expand=True, fill='both')
        self.current_frame = frame

    def on_frame_configure(self, event):
        self.chat_canvas.configure(scrollregion=self.chat_canvas.bbox("all"))

    def connect_to_server(self):
        host = self.server_entry.get()
        try:
            self.sock.connect((host, 2500))
            return True
        except Exception as e:
            messagebox.showerror("연결 오류", f"서버 연결 실패: {str(e)}")
            return False
    def connect_to_server(self):
        host = self.server_entry.get()
        try:
            if self.sock:
                self.sock.close()
            self.sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
            self.sock.connect((host, 2500))
            return True
        except Exception as e:
            messagebox.showerror("연결 오류", f"서버 연결 실패: {str(e)}")
            return False
        
    def login(self):
        if not self.connect_to_server():
            return

        username = self.username_entry.get()
        password = self.password_entry.get()
        self.sock.send(f"LOGIN:{username}:{password}".encode())
        response = self.sock.recv(1024).decode()
        if response == "LOGIN_SUCCESS":
            self.nickname = username
            self.show_frame(self.chat_frame)
            self.display_message(f"{username}님으로 로그인했습니다.", 'left', 'white')
            receive_thread = threading.Thread(target=self.receive_messages)
            receive_thread.daemon = True
            receive_thread.start()
        else:
            messagebox.showerror("로그인 실패", "아이디 또는 비밀번호가 올바르지 않습니다.")
            self.sock.close()
            self.sock = None

    def register(self):
        username = self.username_entry.get()
        password = self.password_entry.get()
        
        if not username or not password:
            messagebox.showerror("입력 오류", "사용자 이름과 비밀번호를 모두 입력해주세요.")
            return
        
        confirm = messagebox.askyesno("계정 생성 확인", f"'{username}' 계정을 생성하시겠습니까?")
        if not confirm:
            return
        
        if not self.connect_to_server():
            return

        self.sock.send(f"REGISTER:{username}:{password}".encode())
        response = self.sock.recv(1024).decode()
        if response == "REGISTER_SUCCESS":
            messagebox.showinfo("회원가입 성공", "회원가입이 완료되었습니다. 로그인해주세요.")
        else:
            messagebox.showerror("회원가입 실패", "이미 존재하는 사용자명입니다.")
        self.sock.close()
        self.sock = None

    def send_message(self, event=None):
        message = self.message_entry.get()
        if message and self.nickname:
            try:
                self.sock.send(message.encode())
                self.message_entry.delete(0, tk.END)
            except:
                self.display_message("메시지 전송 실패", 'left', 'red')

    def receive_messages(self):
        while True:
            try:
                data = self.sock.recv(1024)
                if data:
                    message = data.decode()
                    if message.startswith(f"{self.nickname}:"):
                        self.display_message(message, 'right', '#fee500')
                    else:
                        self.display_message(message, 'left', 'white')
            except:
                self.display_message("서버와의 연결이 끊겼습니다.", 'left', 'red')
                break


    def display_message(self, message, align, bg_color):
        frame = tk.Frame(self.message_frame, bg="#b2c7d9")
        frame.pack(side=tk.TOP, fill=tk.X, padx=2, pady=2)

        time_now = datetime.datetime.now().strftime("%H:%M")
        
        if align == 'left':
            msg_label = tk.Label(frame, text=message, bg=bg_color, fg="black", wraplength=200, justify=tk.LEFT, padx=5, pady=5)
            msg_label.pack(side=tk.LEFT)
            time_label = tk.Label(frame, text=time_now, bg="#b2c7d9", fg="gray", font=("Arial", 7))
            time_label.pack(side=tk.LEFT, padx=(2, 0), pady=(0, 2), anchor='s')
        else:
            inner_frame = tk.Frame(frame, bg="#b2c7d9")
            inner_frame.pack(side=tk.RIGHT)
            
            time_label = tk.Label(inner_frame, text=time_now, bg="#b2c7d9", fg="gray", font=("Arial", 7))
            time_label.pack(side=tk.LEFT, padx=(0, 2), pady=(0, 2), anchor='s')
            
            msg_label = tk.Label(inner_frame, text=message, bg=bg_color, fg="black", wraplength=200, justify=tk.RIGHT, padx=5, pady=5)
            msg_label.pack(side=tk.RIGHT)

        self.chat_canvas.update_idletasks()
        self.chat_canvas.yview_moveto(1.0)



        
if __name__ == "__main__":
    root = tk.Tk()
    client = ChatClient(root)
    root.mainloop()


In [4]:
import socket
import threading
import tkinter as tk
from tkinter import ttk, messagebox
from emoji import emojize

class ChatClient:
    def __init__(self, master):
        self.master = master
        self.master.title("NA_KakaoTalk")
        self.master.geometry("350x500")
        self.master.configure(bg="#b2c7d9")

        self.nickname = None
        self.sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)

        self.create_login_frame()
        self.create_chat_frame()

        self.current_frame = self.login_frame
        self.show_frame(self.login_frame)

        # 이모지 단축키 딕셔너리
        self.emoji_shortcuts = {
            ":smile:": "😊",
            ":heart:": "❤️",
            ":thumbsup:": "👍"
        }

    def create_login_frame(self):
        self.login_frame = tk.Frame(self.master, bg="#b2c7d9")
        
        tk.Label(self.login_frame, text="서버 주소:", bg="#b2c7d9").pack(pady=5)
        self.server_entry = tk.Entry(self.login_frame)
        self.server_entry.insert(0, '127.0.0.1')
        self.server_entry.pack(pady=5)

        tk.Label(self.login_frame, text="사용자 이름:", bg="#b2c7d9").pack(pady=5)
        self.username_entry = tk.Entry(self.login_frame)
        self.username_entry.pack(pady=5)

        tk.Label(self.login_frame, text="비밀번호:", bg="#b2c7d9").pack(pady=5)
        self.password_entry = tk.Entry(self.login_frame, show='*')
        self.password_entry.pack(pady=5)

        self.login_button = ttk.Button(self.login_frame, text="로그인", command=self.login)
        self.login_button.pack(pady=10)

        self.register_button = ttk.Button(self.login_frame, text="회원가입", command=self.register)
        self.register_button.pack(pady=5)

    def create_chat_frame(self):
        self.chat_frame = tk.Frame(self.master, bg="#b2c7d9")

        self.chat_canvas = tk.Canvas(self.chat_frame, bg="#b2c7d9", highlightthickness=0)
        self.chat_canvas.pack(expand=True, fill='both', padx=10, pady=10)

        self.scrollbar = ttk.Scrollbar(self.chat_canvas, orient="vertical", command=self.chat_canvas.yview)
        self.scrollbar.pack(side="right", fill="y")

        self.chat_canvas.configure(yscrollcommand=self.scrollbar.set)

        self.message_frame = tk.Frame(self.chat_canvas, bg="#b2c7d9")
        self.chat_canvas.create_window((50, 0), window=self.message_frame, anchor="nw")

        self.input_frame = tk.Frame(self.chat_frame, bg="#b2c7d9")
        self.input_frame.pack(side=tk.BOTTOM, fill=tk.X, padx=10, pady=10)

        self.message_entry = tk.Entry(self.input_frame, bg="white", fg="black")
        self.message_entry.pack(side=tk.LEFT, expand=True, fill=tk.X, padx=(0, 5))
        self.message_entry.bind("<Return>", self.send_message)

        self.send_button = ttk.Button(self.input_frame, text="전송", command=self.send_message)
        self.send_button.pack(side=tk.RIGHT)

        # 이모지 버튼 추가
        self.emoji_button = ttk.Button(self.input_frame, text="😊", command=self.show_emoji_menu)
        self.emoji_button.pack(side=tk.RIGHT, padx=5)

        self.message_frame.bind("<Configure>", self.on_frame_configure)

    def show_frame(self, frame):
        if self.current_frame:
            self.current_frame.pack_forget()
        frame.pack(expand=True, fill='both')
        self.current_frame = frame

    def on_frame_configure(self, event):
        self.chat_canvas.configure(scrollregion=self.chat_canvas.bbox("all"))

    def connect_to_server(self):
        host = self.server_entry.get()
        try:
            if self.sock:
                self.sock.close()
            self.sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
            self.sock.connect((host, 2500))
            return True
        except Exception as e:
            messagebox.showerror("연결 오류", f"서버 연결 실패: {str(e)}")
            return False
        
    def login(self):
        if not self.connect_to_server():
            return

        username = self.username_entry.get()
        password = self.password_entry.get()
        self.sock.send(f"LOGIN:{username}:{password}".encode())
        response = self.sock.recv(1024).decode()
        if response == "LOGIN_SUCCESS":
            self.nickname = username
            self.show_frame(self.chat_frame)
            self.display_message(f"{username}님으로 로그인했습니다.", 'left', 'white')
            receive_thread = threading.Thread(target=self.receive_messages)
            receive_thread.daemon = True
            receive_thread.start()
        else:
            messagebox.showerror("로그인 실패", "아이디 또는 비밀번호가 올바르지 않습니다.")
            self.sock.close()
            self.sock = None

    def register(self):
        username = self.username_entry.get()
        password = self.password_entry.get()
        
        if not username or not password:
            messagebox.showerror("입력 오류", "사용자 이름과 비밀번호를 모두 입력해주세요.")
            return
        
        confirm = messagebox.askyesno("계정 생성 확인", f"'{username}' 계정을 생성하시겠습니까?")
        if not confirm:
            return
        
        if not self.connect_to_server():
            return

        self.sock.send(f"REGISTER:{username}:{password}".encode())
        response = self.sock.recv(1024).decode()
        if response == "REGISTER_SUCCESS":
            messagebox.showinfo("회원가입 성공", "회원가입이 완료되었습니다. 로그인해주세요.")
        else:
            messagebox.showerror("회원가입 실패", "이미 존재하는 사용자명입니다.")
        self.sock.close()
        self.sock = None

    def send_message(self, event=None):
        message = self.message_entry.get()
        if message and self.nickname:
            # 이모지 단축키 변환
            for shortcut, emoji in self.emoji_shortcuts.items():
                message = message.replace(shortcut, emoji)
            try:
                self.sock.send(message.encode())
                self.message_entry.delete(0, tk.END)
            except:
                self.display_message("메시지 전송 실패", 'left', 'red')

    def receive_messages(self):
        while True:
            try:
                data = self.sock.recv(1024)
                if data:
                    message = data.decode()
                    if message.startswith(f"{self.nickname}:"):
                        self.display_message(message, 'right', '#fee500')
                    else:
                        self.display_message(message, 'left', 'white')
            except:
                self.display_message("서버와의 연결이 끊겼습니다.", 'left', 'red')
                break

    def display_message(self, message, align, bg_color):
        frame = tk.Frame(self.message_frame, bg="#b2c7d9")
        frame.pack(side=tk.TOP, fill=tk.X, padx=2, pady=2)

        if align == 'left':
            frame.pack_configure(anchor='w')
            msg_label = tk.Label(frame, text=message, bg=bg_color, fg="black", wraplength=200, justify=tk.LEFT, padx=5, pady=5)
            msg_label.pack(side=tk.LEFT)
        else:
            frame.pack_configure(anchor='e')
            msg_label = tk.Label(frame, text=message, bg=bg_color, fg="black", wraplength=200, justify=tk.RIGHT, padx=5, pady=5)
            msg_label.pack(side=tk.RIGHT)

        self.chat_canvas.update_idletasks()
        self.chat_canvas.yview_moveto(1.0)

    def show_emoji_menu(self):
        emoji_menu = tk.Menu(self.master, tearoff=0)
        for shortcut, emoji in self.emoji_shortcuts.items():
            emoji_menu.add_command(label=f"{emoji} {shortcut}", command=lambda s=shortcut: self.insert_emoji(s))
        emoji_menu.post(self.emoji_button.winfo_rootx(), self.emoji_button.winfo_rooty() + self.emoji_button.winfo_height())

    def insert_emoji(self, shortcut):
        self.message_entry.insert(tk.END, shortcut)

if __name__ == "__main__":
    root = tk.Tk()
    client = ChatClient(root)
    root.mainloop()


In [2]:
pip install emoji


   ---------------------------------------- 0.0/586.9 kB ? eta -:--:--
    --------------------------------------- 10.2/586.9 kB ? eta -:--:--
   ------------------ --------------------- 276.5/586.9 kB 4.3 MB/s eta 0:00:01
   ---------------------------------------- 586.9/586.9 kB 5.3 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.
